In [1]:
import os
import h5py
import gym
import numpy as np
import json
import torch
import yaml
import hydra
import myosuite.envs.myo.myobase
import omegaconf
from pathlib import Path

MyoSuite:> Registering Myo Envs


## Add task_id to bidex tasks

In [2]:
import sys
import glob

sys.path.append("../")
from robomimic_data import add_task_id

In [3]:
data_paths = glob.glob('/home/krishnans/ngc/DexterousHands/bidexhands/logs/ShadowHand**/**/**/rollout*0.hdf5')
task_names = ["switch"] * 2 + ["scissors"] * 3
task_set = 'bidex'

add_task_id(data_paths, task_names, task_set)

In [ ]:
'''
switch the image channels to (c, h, w) if it is in the order of (h, w, c)
'''
def convert_hwc_to_chw(data):
    if len(data.shape) >= 3 and data.shape[-1] < 5:
        if type(data) == torch.tensor:
            data = torch.moveaxis(data, -1, -3).contiguous()
        elif type(data) == np.ndarray:
            data = np.ascontiguousarray(np.moveaxis(data, -1, -3))
        else:
            raise NotImplementedError
    
    return data


def convert_articulate_to_robosuite(dataset_path, config_path=None):
    config = {}
    if config_path is not None:
        config = yaml.safe_load(open(config_path, "r"))

    with h5py.File(dataset_path, mode="a") as data:
        data.attrs["env_args"] = json.dumps(config)
        for ep in data["data"].keys():
            data["data/{}".format(ep)].attrs["num_samples"] = data["data/{}".format(ep)]["actions"].shape[0]
        # Create next_obs group for every demo in data, and copy obs/{key}.data[1:] to this group
        for demo_key in data["data"].keys():
            demo_group = data["data"][demo_key]
            next_obs_group = demo_group.create_group("next_obs")
            
            for obs_key in demo_group["obs"].keys():
                obs_data = demo_group["obs"][obs_key]
                next_obs_group.create_dataset(obs_key, data=obs_data[1:])

In [ ]:
import robomimic.utils.file_utils as FileUtils

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY

# set download folder and make it
download_folder = "/tmp/robomimic_ds_example"
os.makedirs(download_folder, exist_ok=True)

# download the dataset
task = "lift"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"], 
    download_dir=download_folder,
)

# enforce that the dataset exists
dataset_path = os.path.join(download_folder, "low_dim_v141.hdf5")
assert os.path.exists(dataset_path)

## Read quantities from dataset

Next, let's demonstrate how to read different quantities from the dataset. There are scripts such as `scripts/get_dataset_info.py` that can help you easily understand the contents of a dataset, but in this example, we'll break down how to do this directly.

First, let's take a look at the number of demonstrations in the file.

```python
dataset_path = "/home/krishnans/ngc/policy_learning_toolkit/datasets/articulate_multi_spray_scissors/train/merged.hdf5"
dataset_path = "/home/krishnans/ngc/DexterousHands/policy_learning_toolkit/train/bidex_runs/ShadowHandScissors/ppo/ppo_seed-1/rollouts_1000.hdf5"
```

In [3]:
# Myosuite task dataset paths
dataset_root = Path("/juno/u/ksrini/multi_task_experts/datasets/myosuite")
dataset_path = list(dataset_root.iterdir())[0].absolute()

In [10]:
# open file
f = h5py.File(dataset_path, "r")

# each demonstration is a group under "data"
demos = list(f["data"].keys())
num_demos = len(demos)

print("hdf5 file {} has {} demonstrations".format(dataset_path, num_demos))

hdf5 file /juno/u/ksrini/multi_task_experts/datasets/myosuite/myo-key-turn_100.hdf5 has 100 demonstrations


In [18]:
f.close()

Next, let's list all of the demonstrations, along with the number of state-action pairs in each demonstration.

In [11]:
# each demonstration is named "demo_#" where # is a number.
# Let's put the demonstration list in increasing episode order
inds = np.argsort([int(elem[5:]) for elem in demos])
demos = [demos[i] for i in inds]

for ep in demos:
    num_actions = f["data/{}/actions".format(ep)].shape[0]
    print("{} has {} samples".format(ep, num_actions))

KeyError: "Unable to synchronously open object (object 'actions' doesn't exist)"

Now, let's dig into a single trajectory to take a look at some of the quantities in each demonstration.

In [ ]:
# look at first demonstration
demo_key = demos[0]
demo_grp = f["data/{}".format(demo_key)]

# Each observation is a dictionary that maps modalities to numpy arrays, and
# each action is a numpy array. Let's print the observations and actions for the 
# first 5 timesteps of this trajectory.
for t in range(5):
    print("timestep {}".format(t))
    obs_t = dict()
    # each observation modality is stored as a subgroup
    for k in demo_grp["obs"]:
        obs_t[k] = demo_grp["obs/{}".format(k)][t] # numpy array
    act_t = demo_grp["action"][t]
    
    # pretty-print observation and action using json
    obs_t_pp = { k : obs_t[k].tolist() for k in obs_t }
    print("obs")
    print(json.dumps(obs_t_pp, indent=4))
    print("action")
    print(act_t)

In [13]:
demo_grp['obs']['fixed_camera'][0]

array([[[234, 180, 133],
        [233, 181, 134],
        [233, 181, 134],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[229, 177, 131],
        [231, 178, 132],
        [235, 180, 133],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[233, 177, 131],
        [233, 177, 130],
        [236, 178, 130],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[230, 167, 118],
        [231, 168, 119],
        [232, 169, 120],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[236, 175, 125],
        [236, 174, 124],
        [236, 173, 124],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[228, 165, 116],
        [227, 163, 114],
        [226, 161, 112],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [14]:
# we can also grab multiple timesteps at once directly, or even the full trajectory at once
first_ten_actions = demo_grp["actions"][:10]
print("shape of first ten actions {}".format(first_ten_actions.shape))
all_actions = demo_grp["actions"][:]
print("shape of all actions {}".format(all_actions.shape))

KeyError: "Unable to synchronously open object (object 'actions' doesn't exist)"

# Dataset helpers

In [2]:
import h5py, yaml

def merge_hdf5_files(source_files, target_file):
    """
    Merge multiple HDF5 files into a single file, ensuring that "data/demo_X" keys
    are unique across the merged file.

    Args:
        source_files (list of str): List of paths to source HDF5 files to merge.
        target_file (str): Path to the target HDF5 file to create.
    """
    with h5py.File(target_file, 'w') as target_h5:
        if "data" not in target_h5.keys():
            target_h5.create_group("data")  # Ensure the 'data' group exists in the target file
        for source_file in source_files:
            with h5py.File(source_file, 'r') as source_h5:
                # Iterate over each "demo_X" group in the source file
                for key in source_h5['data'].keys():
                    # Extract the demo number from the key and increment it by the current offset
                    new_demo_key = f'demo_{len(target_h5["data"].keys())}'

                    # Copy the group to the new file with the updated demo key
                    source_h5.copy(f'data/{key}', target_h5["data"], new_demo_key)


def convert_camera_obs_hwc(source_file, target_file):
    """
    Convert all observations with the name "camera" in the key from (c, h, w) to (h, w, c) shape ordering,
    and change dtype to np.uint8 for a single source file, and map it to a target file.

    Args:
        source_file (str): Path to the source HDF5 file.
        target_file (str): Path to the target HDF5 file to create.
    """
    with h5py.File(source_file, 'r') as source_h5:
        with h5py.File(target_file, 'w') as target_h5:
            # Iterate over each "demo_X" group in the source file
            for demo_key in source_h5['data'].keys():
                demo_group = source_h5['data'][demo_key]
                target_demo_group = target_h5.require_group(f'data/{demo_key}')
                
                # Check if the "obs" group exists in the demo group
                for key in demo_group:
                    if key == "obs":
                        obs_group = demo_group["obs"]
                        # Add "obs" group to the target_demo_group if it doesn't already exist
                        target_demo_group.create_group("obs")
                        # Iterate over each item in the "obs" group
                        for item_key in obs_group.keys():
                            item = obs_group[item_key]
                            
                            # Check if the item is an observation with "camera" in the key
                            if "camera" in item_key and len(item.shape) == 4:  # Assuming shape is (timesteps, c, h, w)
                                # Convert (c, h, w) to (h, w, c) and change dtype to np.uint8
                                converted_item = item[:].transpose(0, 2, 3, 1).astype(np.uint8)
                                target_demo_group["obs"].create_dataset(item_key, data=converted_item)
                            else:
                                # For other items within "obs", just copy them as they are
                                obs_group.copy(item_key, target_demo_group["obs"])
                    else:
                        demo_group.copy(f'{key}', target_demo_group, key)
                        # target_demo_group.create_dataset(key, data=demo_group[key])


def convert_articulate_to_robosuite(dataset_path, config_path=None):
    config = {}
    if config_path is not None:
        config = yaml.safe_load(open(config_path, "r"))

    with h5py.File(dataset_path, mode="a") as data:
        data.attrs["env_args"] = json.dumps(config)
        for ep in data["data"].keys():
            data["data/{}".format(ep)].attrs["num_samples"] = data["data/{}".format(ep)]["actions"].shape[0]

MYOSUITE_TASKS = {
	'myo-reach': 'myoHandReachFixed-v0',
	'myo-reach-hard': 'myoHandReachRandom-v0',
	'myo-pose': 'myoHandPoseFixed-v0',
	'myo-pose-hard': 'myoHandPoseRandom-v0',
	'myo-obj-hold': 'myoHandObjHoldFixed-v0',
	'myo-obj-hold-hard': 'myoHandObjHoldRandom-v0',
	'myo-key-turn': 'myoHandKeyTurnFixed-v0',
	'myo-key-turn-hard': 'myoHandKeyTurnRandom-v0',
	'myo-pen-twirl': 'myoHandPenTwirlFixed-v0',
	'myo-pen-twirl-hard': 'myoHandPenTwirlRandom-v0',
}

def add_mask(dataset_paths):
    """Adds a mask to the dataset to indicate which demonstrations are successful.
    A demonstration is successful if it has at least one sample."""
    for dataset_path in dataset_paths:
        cfg = omegaconf.OmegaConf.load(dataset_paths[0].parent/'.hydra/config.yaml')
        min_ep_len = gym.envs.registry.spec(MYOSUITE_TASKS[cfg.task]).max_episode_steps + 1
        with h5py.File(dataset_path, "a") as data:
            success_demos = []
            for ep in data["data"].keys():
                        if data[f"data/{ep}"].attrs['num_samples'] >= 1:
                            success_demos.append(ep.encode('utf-8'))  # HDF5 requires bytes for strings
            if "mask" not in data.keys():
                data.create_group("mask")
            if "traj_success" not in data["mask"].keys():
                data["mask"].create_dataset("traj_success", data=np.array(success_demos, dtype=h5py.string_dtype(encoding='utf-8')))
            else:
                print(f"skipping mask for {dataset_path}, traj_success already exists")

In [11]:
with h5py.File(dataset_path, "r") as data:
    print([s.decode('utf-8') for s in data['mask']['traj_success']])

['demo_0', 'demo_1', 'demo_10', 'demo_11', 'demo_12', 'demo_13', 'demo_14', 'demo_15', 'demo_16', 'demo_17', 'demo_18', 'demo_19', 'demo_2', 'demo_20', 'demo_21', 'demo_22', 'demo_23', 'demo_24', 'demo_25', 'demo_26', 'demo_27', 'demo_28', 'demo_29', 'demo_3', 'demo_30', 'demo_31', 'demo_32', 'demo_33', 'demo_34', 'demo_35', 'demo_36', 'demo_37', 'demo_38', 'demo_39', 'demo_4', 'demo_40', 'demo_41', 'demo_42', 'demo_43', 'demo_44', 'demo_45', 'demo_46', 'demo_47', 'demo_48', 'demo_49', 'demo_5', 'demo_50', 'demo_51', 'demo_52', 'demo_53', 'demo_54', 'demo_55', 'demo_56', 'demo_57', 'demo_58', 'demo_59', 'demo_6', 'demo_60', 'demo_61', 'demo_62', 'demo_63', 'demo_64', 'demo_65', 'demo_66', 'demo_67', 'demo_68', 'demo_69', 'demo_7', 'demo_70', 'demo_71', 'demo_72', 'demo_73', 'demo_74', 'demo_75', 'demo_76', 'demo_77', 'demo_78', 'demo_79', 'demo_8', 'demo_80', 'demo_81', 'demo_82', 'demo_83', 'demo_84', 'demo_85', 'demo_86', 'demo_87', 'demo_88', 'demo_89', 'demo_9', 'demo_90', 'demo_91

In [12]:
ksrini_hdf5_dir = Path('/juno/u/ksrini/multi_task_experts/collect_myosuite/multirun/2024-05-19')
henryang_hdf5_dir = Path('/juno/u/henryang/multi_task_experts/collect_myosuite/outputs/2024-05-18')
k_paths = list(ksrini_hdf5_dir.rglob(pattern='*.hdf5'))
h_paths = list(henryang_hdf5_dir.rglob(pattern='*.hdf5'))


In [14]:
add_mask(h_paths)

skipping mask for /juno/u/henryang/multi_task_experts/collect_myosuite/outputs/2024-05-18/17-25-38/myo-reach_100.hdf5, traj_success already exists
skipping mask for /juno/u/henryang/multi_task_experts/collect_myosuite/outputs/2024-05-18/17-09-21/myo-key-turn_100.hdf5, traj_success already exists


In [17]:
add_mask(k_paths)

skipping mask for /juno/u/ksrini/multi_task_experts/collect_myosuite/multirun/2024-05-19/00-07-27/0/myo-obj-hold-hard_500.hdf5, traj_success already exists


In [11]:
ls -al /juno/u/ksrini/multi_task_experts/collect_myosuite/multirun/2024-05-19/00-07-27/0/myo-obj-hold-hard_500.hdf5

-rw-r--r-- 1 ksrini iprl 16831859452 May 19 02:20 /juno/u/ksrini/multi_task_experts/collect_myosuite/multirun/2024-05-19/00-07-27/0/myo-obj-hold-hard_500.hdf5


In [ ]:
robo_formatted_file = dataset_path #  os.path.join(os.path.split(dataset_path)[0], "merge_robosuite.hdf5")

In [ ]:
with h5py.File(robo_formatted_file) as data:
    for ep in data["data"].keys():
        print(f"{ep} num samples", data[f"data/{ep}"].attrs["num_samples"])

## Loading and running bidex task

In [ ]:
import isaacgym, isaacgymenvs
from omegaconf import OmegaConf
from hydra import compose, initialize_config_dir
from isaacgymenvs.utils.rlgames_utils import get_rlgames_env_creator
from isaacgymenvs.utils.reformat import omegaconf_to_dict

def get_expert_cfg(config_path=None, checkpoint_path=None):
    if checkpoint_path:
        config_path = os.path.join(os.path.dirname(os.path.dirname(checkpoint_path)), "config.yaml")
    with open(config_path, "r") as f:
        cfg = OmegaConf.load(config_path)
    return cfg

cfg_expert = get_expert_cfg("/mnt/ws-dmanip/isaacgymenvs/isaacgymenvs/runs/ArticulateSpray2Expert_12-20-34-27/config.yaml")

create_rlgpu_env = get_rlgames_env_creator(
    cfg_expert['seed'],
    omegaconf_to_dict(cfg_expert["task"]),
    cfg_expert["task_name"],
    cfg_expert["sim_device"],
    cfg_expert["rl_device"],
    cfg_expert['graphics_device_id'],
    cfg_expert['headless'],
    multi_gpu=cfg_expert['multi_gpu'],
    )

env = create_rlgpu_env()



In [ ]:
dataset_dir = os.path.dirname(dataset_path)
source_files = list(filter(lambda x: not x.startswith("merged"), map(lambda x: os.path.join(dataset_dir, x), os.listdir(dataset_dir))))
# merge all hdf5 files in dataset_path into a single hdf5 file
merged_dataset_path = os.path.join(os.path.dirname(dataset_path), "merged.hdf5")
if not os.path.exists(merged_dataset_path):
    # os.remove(merged_dataset_path)
    merge_hdf5_files(source_files, merged_dataset_path)

In [ ]:
with h5py.File(merged_dataset_path, "r") as f:
    print("merged dataset has {} demonstrations".format(len(f["data"].keys())))

In [ ]:
# open file
f = h5py.File(merged_dataset_path, "a")

# each demonstration is a group under "data"
demos = list(f["data"].keys())
num_demos = len(demos)
demo_grp = f["data"][demos[0]]

In [ ]:
obs_keys = demo_grp["obs"].keys()
# demo_grp.create_group("next_obs")
for k in obs_keys:
    demo_grp.create_dataset(f"next_obs/{k}", data=demo_grp[f"obs/{k}"][1:])

In [ ]:
# the trajectory also contains the next observations under "next_obs", 
# for convenient use in a batch (offline) RL pipeline. Let's verify
# that "next_obs" and "obs" are offset by 1.
for k in demo_grp["obs"]:
    # obs_{t+1} == next_obs_{t}
    assert(np.allclose(demo_grp["obs"][k][1:], demo_grp["next_obs"][k][:-1]))
print("success")

In [ ]:
convert_articulate_to_robosuite(merged_dataset_path)

In [ ]:
# we also have "done" and "reward" information stored in each trajectory.
# In this case, we have sparse rewards that indicate task completion at
# that timestep.
dones = demo_grp["dones"][:]
rewards = demo_grp["rewards"][:]
print("dones")
print(dones)
print("")
print("rewards")
print(rewards)

In [ ]:
# each demonstration also contains metadata
num_samples = demo_grp.attrs["num_samples"] # number of samples in this trajectory
mujoco_xml_file = demo_grp.attrs["model_file"] # mujoco XML file for this demonstration
print(mujoco_xml_file)

Finally, let's take a look at some global metadata present in the file. The hdf5 file stores environment metadata which is a convenient way to understand which simulation environment (task) the dataset was collected on. 

In [ ]:
env_meta = json.loads(f["data"].attrs["env_args"])
# note: we could also have used the following function:
# env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path=dataset_path)
print("==== Env Meta ====")
print(json.dumps(env_meta, indent=4))
print("")

## Visualizing demonstration trajectories

Finally, let's play some of these demonstrations back in the simulation environment to easily visualize the data that was collected.

It turns out that the environment metadata stored in the hdf5 allows us to easily create a simulation environment that is consistent with the way the dataset was collected!

In [ ]:
import robomimic.utils.env_utils as EnvUtils

# create simulation environment from environment metedata
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta, 
    render=False,            # no on-screen rendering
    render_offscreen=True,   # off-screen rendering to support rendering video frames
)

In [ ]:
import robomimic.utils.obs_utils as ObsUtils

# We normally need to make sure robomimic knows which observations are images (for the
# data processing pipeline). This is usually inferred from your training config, but
# since we are just playing back demonstrations, we just need to initialize robomimic
# with a dummy spec.
dummy_spec = dict(
    obs=dict(
            low_dim=["robot0_eef_pos"],
            rgb=[],
        ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)

In [ ]:
import imageio

# prepare to write playback trajectories to video
video_path = os.path.join(download_folder, "playback.mp4")
video_writer = imageio.get_writer(video_path, fps=20)

In [ ]:
def playback_trajectory(demo_key):
    """
    Simple helper function to playback the trajectory stored under the hdf5 group @demo_key and
    write frames rendered from the simulation to the active @video_writer.
    """
    
    # robosuite datasets store the ground-truth simulator states under the "states" key.
    # We will use the first one, alone with the model xml, to reset the environment to
    # the initial configuration before playing back actions.
    init_state = f["data/{}/states".format(demo_key)][0]
    model_xml = f["data/{}".format(demo_key)].attrs["model_file"]
    initial_state_dict = dict(states=init_state, model=model_xml)
    
    # reset to initial state
    env.reset_to(initial_state_dict)
    
    # playback actions one by one, and render frames
    actions = f["data/{}/actions".format(demo_key)][:]
    for t in range(actions.shape[0]):
        env.step(actions[t])
        video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
        video_writer.append_data(video_img)

In [ ]:
# playback the first 5 demos
for ep in demos[:5]:
    print("Playing back demo key: {}".format(ep))
    playback_trajectory(ep)

# done writing video
video_writer.close()

In [ ]:
# view the trajectories!
from IPython.display import Video
Video(video_path, embed=True)